<a href="https://colab.research.google.com/github/oluwap3lumi/30DayMapChallenge/blob/main/TRAIN_SEGMENTATION_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.8/242.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.2/589.2 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.3/605.3 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
 import geoai

In [5]:
train_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_rgb_train.tif"
)
train_vector_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train_buildings.geojson"
test_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_test.tif"
)

In [6]:
train_raster_path = geoai.download_file(train_raster_url)
train_vector_path = geoai.download_file(train_vector_url)
test_raster_path = geoai.download_file(test_raster_url)

In [7]:
geoai.get_raster_info(train_raster_path)

{'driver': 'GTiff',
 'width': 2503,
 'height': 1126,
 'count': 3,
 'dtype': 'uint8',
 'crs': 'EPSG:26911',
 'transform': Affine(0.6000000000000046, 0.0, 454780.8,
        0.0, -0.6, 5278242.6),
 'bounds': BoundingBox(left=454780.8, bottom=5277567.0, right=456282.6, top=5278242.6),
 'resolution': (0.6000000000000046, 0.6),
 'nodata': None,
 'band_stats': [{'band': 1,
   'min': 12.0,
   'max': 251.0,
   'mean': 150.6730747259594,
   'std': 48.01908734374099},
  {'band': 2,
   'min': 49.0,
   'max': 251.0,
   'mean': 141.92468895229808,
   'std': 43.46595463573497},
  {'band': 3,
   'min': 53.0,
   'max': 251.0,
   'mean': 120.89909373405554,
   'std': 41.78086244480775}]}

In [12]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_url)

In [9]:
geoai.view_raster(test_raster_path)

Map(center=[47.6464835, -117.59043650000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

In [13]:
out_folder = "buildings"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)


Raster info for naip_rgb_train.tif:
  CRS: EPSG:26911
  Dimensions: 2503 x 1126
  Resolution: (0.6000000000000046, 0.6)
  Bands: 3
  Bounds: BoundingBox(left=454780.8, bottom=5277567.0, right=456282.6, top=5278242.6)
Loaded 735 features from naip_train_buildings.geojson
Vector CRS: EPSG:4326
Reprojecting features from EPSG:4326 to EPSG:26911
Found 1 unique classes: ['building']


Generated: 36, With features: 36: 100%|██████████| 36/36 [00:07<00:00,  4.69it/s]


------- Export Summary -------
Total tiles exported: 36
Tiles with features: 36 (100.0%)
Average feature pixels per tile: 46795.0
Output saved to: buildings

------- Georeference Verification -------


In [14]:
# Train U-Net model
geoai.train_segmentation_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/unet_models",
    architecture="unet",
    encoder_name="resnet34",
    encoder_weights="imagenet",
    num_channels=3,
    num_classes=2,  # background and building
    batch_size=8,
    num_epochs=5,
    learning_rate=0.001,
    val_split=0.2,
    verbose=True,
)

Using device: cuda
Found 36 image files and 36 label files
Training on 28 images, validating on 8 images
Checking image sizes for compatibility...
All sampled images have the same size: (512, 512)
No resizing needed.
Testing data loader...
Data loader test passed.


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

Starting training with unet + resnet34
Model parameters: 24,436,514
Epoch: 1, Batch: 1/4, Loss: 0.5932, Time: 2.60s
Epoch 1/5: Train Loss: 0.4549, Val Loss: 0.6711, Val IoU: 0.4231, Val Dice: 0.4705
Saving best model with IoU: 0.4231
Epoch: 2, Batch: 1/4, Loss: 0.3020, Time: 1.37s
Epoch 2/5: Train Loss: 0.2594, Val Loss: 3.5186, Val IoU: 0.4270, Val Dice: 0.4735
Saving best model with IoU: 0.4270
Epoch: 3, Batch: 1/4, Loss: 0.1966, Time: 1.69s
Epoch 3/5: Train Loss: 0.1697, Val Loss: 2.4491, Val IoU: 0.4441, Val Dice: 0.5018
Saving best model with IoU: 0.4441
Epoch: 4, Batch: 1/4, Loss: 0.1370, Time: 1.26s
Epoch 4/5: Train Loss: 0.1422, Val Loss: 3.6334, Val IoU: 0.5679, Val Dice: 0.6705
Saving best model with IoU: 0.5679
Epoch: 5, Batch: 1/4, Loss: 0.1181, Time: 1.25s
Epoch 5/5: Train Loss: 0.1164, Val Loss: 1.4208, Val IoU: 0.6981, Val Dice: 0.7926
Saving best model with IoU: 0.6981
Training complete! Best IoU: 0.6981
Models saved to buildings/unet_models


In [15]:
geoai.train_segmentation_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/segformer_models",
    architecture="segformer",
    encoder_name="resnet152",
    encoder_weights="imagenet",
    num_channels=3,
    num_classes=2,
    batch_size=6,  # Smaller batch size for more complex model
    num_epochs=5,
    learning_rate=0.0005,
    val_split=0.2,
)

Using device: cuda
Found 36 image files and 36 label files
Training on 28 images, validating on 8 images
Checking image sizes for compatibility...
All sampled images have the same size: (512, 512)
No resizing needed.
Testing data loader...
Data loader test passed.


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Starting training with segformer + resnet152
Model parameters: 59,473,218
Epoch: 1, Batch: 1/5, Loss: 0.7512, Time: 2.04s
Epoch 1/5: Train Loss: 0.6022, Val Loss: 1.0883, Val IoU: 0.4637, Val Dice: 0.6047
Saving best model with IoU: 0.4637
Epoch: 2, Batch: 1/5, Loss: 0.3713, Time: 1.74s
Epoch 2/5: Train Loss: 0.2857, Val Loss: 1.3229, Val IoU: 0.4844, Val Dice: 0.6100
Saving best model with IoU: 0.4844
Epoch: 3, Batch: 1/5, Loss: 0.2115, Time: 2.40s
Epoch 3/5: Train Loss: 0.1936, Val Loss: 0.2393, Val IoU: 0.6430, Val Dice: 0.7516
Saving best model with IoU: 0.6430
Epoch: 4, Batch: 1/5, Loss: 0.1763, Time: 1.63s
Epoch 4/5: Train Loss: 0.1534, Val Loss: 0.2187, Val IoU: 0.7325, Val Dice: 0.8349
Saving best model with IoU: 0.7325
Epoch: 5, Batch: 1/5, Loss: 0.1256, Time: 1.84s
Epoch 5/5: Train Loss: 0.1294, Val Loss: 0.1573, Val IoU: 0.7809, Val Dice: 0.8690
Saving best model with IoU: 0.7809
Training complete! Best IoU: 0.7809
Models saved to buildings/segformer_models


In [16]:
# Define paths
masks_path = "naip_test_semantic_prediction.tif"
model_path = f"{out_folder}/unet_models/best_model.pth"

In [17]:
# Run semantic segmentation inference
geoai.semantic_segmentation(
    input_path=test_raster_path,
    output_path=masks_path,
    model_path=model_path,
    architecture="unet",
    encoder_name="resnet34",
    num_channels=3,
    num_classes=2,
    window_size=512,
    overlap=256,
    batch_size=4,
)

Input file format: GeoTIFF (.tif)
Processing 65 windows...


84it [00:02, 38.74it/s]


Predicted classes: 2 classes, Background: 91.3%
Inference completed in 2.81 seconds
Saved prediction to naip_test_semantic_prediction.tif


In [18]:
# Run inference with probability output
probability_path = "naip_test_probability_map.tif"

geoai.semantic_segmentation(
    input_path=test_raster_path,
    output_path=masks_path,
    model_path=model_path,
    architecture="unet",
    encoder_name="resnet34",
    num_channels=3,
    num_classes=2,
    window_size=512,
    overlap=256,
    batch_size=4,
    probability_path=probability_path,  # Output probability map
)

Input file format: GeoTIFF (.tif)
Processing 65 windows...


84it [00:02, 38.40it/s]


Predicted classes: 2 classes, Background: 91.3%
Inference completed in 2.76 seconds
Saved prediction to naip_test_semantic_prediction.tif
Saved probability map to naip_test_probability_map.tif


In [19]:
# Visualize probability map for building class (band 2)
geoai.view_raster(
    probability_path, indexes=[2], basemap=test_raster_path, backend="ipyleaflet"
)

Map(center=[47.6464835, -117.59043650000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

In [20]:
# Run inference with custom probability threshold
masks_path_threshold = "naip_test_semantic_prediction_threshold2.tif"

geoai.semantic_segmentation(
    input_path=test_raster_path,
    output_path=masks_path_threshold,
    model_path=model_path,
    architecture="unet",
    encoder_name="resnet34",
    num_channels=3,
    num_classes=2,
    window_size=512,
    overlap=256,
    batch_size=4,
    probability_threshold=0.3,  # Only classify as building if probability >= 0.7
)

Input file format: GeoTIFF (.tif)
Processing 65 windows...


84it [00:02, 38.67it/s]


Using probability threshold: 0.3
Predicted classes: 2 classes, Background: 88.6%
Inference completed in 2.71 seconds
Saved prediction to naip_test_semantic_prediction_threshold2.tif
